In [1]:
!pip install -q tflite-support
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!sudo apt-get install libportaudio2
!pip install sounddevice

     |████████████████████████████████| 42.5 MB 1.4 MB/s 
     |████████████████████████████████| 1.0 MB 37.1 MB/s 
     |████████████████████████████████| 213 kB 33.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
     |████████████████████████████████| 642 kB 4.4 MB/s 
     |████████████████████████████████| 3.4 MB 39.5 MB/s 
     |████████████████████████████████| 6.4 MB 34.5 MB/s 
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |████████████████████████████████| 10.9 MB 20.1 MB/s 
     |████████████████████████████████| 120 kB 48.7 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 1.2 MB 45.7 MB/s 
     |████████████████████████████████| 237 kB 43.5 MB/s 
     |███████████

In [2]:
import numpy as np
import os
import datetime as dt

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
from tensorflow.keras import layers
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Here we can download the train and validate dataset from the dropbox and upload it to google drive and the next cell would connect it

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
# train_data = object_detector.DataLoader.from_pascal_voc(
#     '/content/drive/MyDrive/Colab Notebooks/mbuna/Train',
#     '/content/drive/MyDrive/Colab Notebooks/mbuna/Train',
#     ['Fish', 'Pipe']
# )

# val_data = object_detector.DataLoader.from_pascal_voc(
#     '/content/drive/MyDrive/Colab Notebooks/mbuna/Validate',
#     '/content/drive/MyDrive/Colab Notebooks/mbuna/Validate',
#     ['Fish', 'Pipe']
# )

In [6]:
def tfrecord_count(containing_dir):
    count = 0
    for tfrecord_path in glob(os.path.join(containing_dir, '*.tfrecord')):
        count += sum(1 for _ in tf.data.TFRecordDataset(tfrecord_path))
    return count

In [7]:
from glob import glob
print(tfrecord_count('/content/drive/MyDrive/Colab Notebooks/mbuna/train'))

5754


In [ ]:
train_data = object_detector.DataLoader('/content/drive/MyDrive/Colab Notebooks/mbuna/train/*.tfrecord', 5754, ['Fish', 'Pipe'])
val_data = object_detector.DataLoader('/content/drive/MyDrive/Colab Notebooks/mbuna/valid/*.tfrecord', 479, ['Fish', 'Pipe'])

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 100`, which means it will go through the training dataset 100 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 8 or 16` here so you will see that it takes 239 steps to go through the 1912 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=64, train_whole_model=True, epochs=75, validation_data=val_data, do_train=True)

Epoch 1/75
89/89 [==============================] - 145s 1s/step - det_loss: 0.9320 - cls_loss: 0.5496 - box_loss: 0.0076 - reg_l2_loss: 0.0631 - loss: 0.9951 - learning_rate: 0.0440 - gradient_norm: 0.8151 - val_det_loss: 0.8682 - val_cls_loss: 0.4077 - val_box_loss: 0.0092 - val_reg_l2_loss: 0.0631 - val_loss: 0.9313
Epoch 2/75
89/89 [==============================] - 110s 1s/step - det_loss: 0.3420 - cls_loss: 0.2022 - box_loss: 0.0028 - reg_l2_loss: 0.0632 - loss: 0.4052 - learning_rate: 0.0799 - gradient_norm: 1.0285 - val_det_loss: 0.5422 - val_cls_loss: 0.2823 - val_box_loss: 0.0052 - val_reg_l2_loss: 0.0633 - val_loss: 0.6054
Epoch 3/75
89/89 [==============================] - 109s 1s/step - det_loss: 0.2796 - cls_loss: 0.1721 - box_loss: 0.0021 - reg_l2_loss: 0.0633 - loss: 0.3428 - learning_rate: 0.0798 - gradient_norm: 0.7982 - val_det_loss: 0.4787 - val_cls_loss: 0.2904 - val_box_loss: 0.0038 - val_reg_l2_loss: 0.0632 - val_loss: 0.5420
Epoch 4/75
89/89 [===================

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
filename = f'efficientdet_{dt.date.today().strftime("%m%d%y")}.tflite'
model.export(export_dir='.', tflite_filename=filename)


### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
filename = f'efficientdet_{dt.date.today().strftime("%m%d%y")}.tflite'
model.evaluate_tflite(filename, val_data)

NameError: ignored

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download(filename)

## Compile the model for EdgeTPU

Finally, we'll compile the model using `edgetpu_compiler` so that the model can run on [Google Coral EdgeTPU](https://coral.ai/).

We start with installing the EdgeTPU compiler on Colab.

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

**Note:** When training the model using a custom dataset, beware that if your dataset includes more than 20 classes, you'll probably have slower inference speeds compared to if you have fewer classes. This is due to an aspect of the EfficientDet architecture in which a certain layer cannot compile for the Edge TPU when it carries more than 20 classes.

Before compiling the `.tflite` file for the Edge TPU, it's important to consider whether your model will fit into the Edge TPU memory. 

The Edge TPU has approximately 8 MB of SRAM for [caching model paramaters](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching), so any model close to or over 8 MB will not fit onto the Edge TPU memory. That means the inference times are longer, because some model parameters must be fetched from the host system memory.

One way to elimiate the extra latency is to use [model pipelining](https://coral.ai/docs/edgetpu/pipeline/), which splits the model into segments that can run on separate Edge TPUs in series. This can significantly reduce the latency for big models.

The following table provides recommendations for the number of Edge TPUs to use with each EfficientDet-Lite model.

| Model architecture | Minimum TPUs | Recommended TPUs
|--------------------|-------|-------|
| EfficientDet-Lite0 | 1     | 1     |
| EfficientDet-Lite1 | 1     | 1     |
| EfficientDet-Lite2 | 1     | 2     |
| EfficientDet-Lite3 | 2     | 2     |
| EfficientDet-Lite4 | 2     | 3     |

If you need extra Edge TPUs for your model, then update `NUMBER_OF_TPUS` here:

In [ ]:
NUMBER_OF_TPUS = 1

!edgetpu_compiler $filename --num_segments=$NUMBER_OF_TPUS

In [ ]:
!pip install -q tflite-support

Finally, we'll copy the metadata, including the label file, from the original TensorFlow Lite model to the EdgeTPU model.

In [ ]:
from tflite_support import metadata

edge_filename = f'efficientdet_{dt.date.today().strftime("%m%d%y")}_edgetpu.tflite'
populator_dst = metadata.MetadataPopulator.with_model_file(edge_filename)

with open(filename, 'rb') as f:
  populator_dst.load_metadata_and_associated_files(f.read())

populator_dst.populate()
updated_model_buf = populator_dst.get_model_buffer()

In [ ]:
# Download the TFLite model compiled for EdgeTPU to your local computer.
from google.colab import files
files.download(edge_filename)